In [ ]:
import numpy as np
from scipy import special
import matplotlib.pyplot as plt

In [1]:
def t_dist(tau,t,t_mid):
    #distributes the angle as a function of time from pi to 0
    delta_t = t - t_mid
    return np.pi*(1 - special.erf(delta_t/tau))/2

def var_calc(t9,nn):
    #calculates the variables required to parametrize the trajectory
    A= np.array([[-1,0,1],[1,0,1],[0,-1,1]])
    
    x = np.linalg.solve(A,t9)
    rx_cos = x[0]
    ry_sin = x[1]
    x_0 = x[2]

    x = np.linalg.solve(A,nn)
    rx_sin = x[0]
    ry_cos = x[1]
    y_0 = x[2]

    return [x_0,y_0,rx_cos,ry_sin,rx_sin,ry_cos]

def pulse(varia,t,param_list,time_func):
    if 'total_points' not in param_list:
        time = np.linspace(t[0],t[1])
    else:
        time = np.linspace(t[0],t[1],param_list['pulse_size'])

    theta = time_func(param_list['tau'],time,t[1]/2)

    x_t = varia[2]*np.cos(theta) - varia[3]*np.sin(theta) + varia[0]
    y_t = varia[4]*np.cos(theta) - varia[5]*np.sin(theta) + varia[1]
        
    return x_t,y_t,time
    
def pulse_return(varia,t, param_list, t9,nn, time_func,t9_return = None):

    if t9_return is None: t9_return = t9[0]

    t_pulse = param_list['pulse_frac']*t[1]
    t_return_start = t_pulse*(1 + 1e-3)
    t_peak = t_pulse/2
    
    slope = (nn[1] - nn[0])/(t9[1] - t9[0])
    inter = nn[1] - slope*t9[1]

    x_t,y_t,time = pulse(varia,[t[0],t_pulse],param_list,time_func)

    time = np.append(time,np.linspace(t_return_start,t[1],param_list['return_size']))
    x_t = np.append(x_t,np.linspace(t9[1],t9_return),param_list['return_size'])
    y_t = np.append(y_t,slope*np.linspace(t9[1],t9_return,param_list['return_size']) + inter)
    
    return x_t,y_t,time

def pulse_return_loop(varia,t,param_list,t9,nn,time_func):

    slope_vert = (nn[2] - nn[0])/(t9[2] - t9[0])
    slope_hori = (nn[1] - nn[0])/(t9[1] - t9[0])

    inter_vert = nn[0] - slope_vert*t9[0]
    inter_hori = nn[0] - slope_hori*t9[0]

    dist_vert = ((nn[2] - nn[0])**2 + (t9[2] - t9[0])**2)**(1/2)
    dist_hori = ((nn[1] - nn[0])**2 + (t9[1] - t9[0])**2)**(1/2)

    t_loop = t[1]/param_list['n_loops']

    t9_return = t9[0] + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)

    x_t,y_t,time = pulse_return(varia,[t[0],t_loop],param_list,t9,nn,time_func,t9_return)

    for i in range(1, param_list['n_loops']):
        
        nn_return = slope_hori*t9_return + inter_hori

        t9_2 = t9_return + dist_hori/(1 + slope_hori**2)**(1/2)
        nn_2 = slope_hori*t9_2 + inter_hori

        t9_3 = t9_return + (1 + param_list['peak_scale'])**(i) * dist_vert/(1 + slope_vert**2)**(1/2)
        b = nn_return - slope_vert*t9_return
        nn_3 = slope_vert*t9_3 + b

        varia = var_calc([t9_return,t9_2,t9_3],[nn_return,nn_2,nn_3])
        x_ti,y_ti,time_i = pulse_return(varia,[t[0],t_loop],param_list,[t9_return,t9_2,t9_3],[nn_return,nn_2,nn_3],time_func,t9_return)
        time = np.append(time, i*t_loop + time_i)
        t9_return = t9_return + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)
        
    return x_t,y_t,time

def traj_builder(t,t9,nn,param_list,traj = 'pulse',time_func = t_dist):
    #builds the trajectories and saves to file
    #t = 2x1 array of start and end time
    #t9 = 3x1 array containing start, end, and peak t9 (in that order)
    #nn = 3x1 array containing the power of 10 of the start, end, and peak of neutron number density (in that order)
    
    varia = var_calc(t9,nn)
    
    if traj == 'pulse':
        x_t,y_t,time = pulse(varia, t, param_list,time_func)
        
    elif traj == 'pulse_return':
        x_t, y_t,time = pulse_return(varia,t,param_list,t9,nn,time_func)

    elif traj == 'pulse_return_loop':
        x_t,y_t,time = pulse_return_loop(varia,t,param_list,t9,nn,time_func)

    return x_t,y_t,time

def param_builder():
    return

def write_to_files():
    return

In [ ]:
#setup the parameters to build your trajectory

#temperature in billions of kelvin
t9_1 = #start temperature
t9_2 = #end temperature
t9_3 = #temperature at peak n_n
####################################

#the power of 10 of neutron number density
nn_1 = #start
nn_2 = #end
nn_3 = #peak
###########################################

#time
